In [209]:
import re
import numpy as np
import pandas as pd
import random
import csv
import subprocess

In [210]:
# List of parameters to search for
parameters = [
    'toxe_n', 'toxm_n', 'toxref_n', 'toxe_p', 'toxm_p',
    'toxref_p', 'toxp', 'xj_n', 'xj_p', 'ndep_n', 'ndep_p'
]

In [211]:
# Function to extract parameter values from the file
def extract_parameter_value(file_content, parameter):
    match = re.search(rf'\b{parameter}\s*=\s*([\d.e+-]+)', file_content)
    if match:
        return match.group(0), float(match.group(1))
    else:
        return None, None

In [212]:
# Function to calculate mean and standard deviation for parameters
def calculate_mean_std(file_path, parameter):
    with open(file_path, 'r') as file:
        content = file.read()
    _, mean = extract_parameter_value(content, parameter)
    std = mean / 30  # Assuming a standard deviation of 1/30th of the mean
    return mean, std

In [213]:
# Function to generate random values within specified ranges
def generate_random_values():
    # Temperature range from -55 to 125 Celsius
    temp = random.uniform(-55, 125)
    # Supply voltage with ±10% variations
    supply = random.uniform(0.9, 1.1)
    # Cqload variations from 0.01f to 5f
    cqload = random.uniform(0.01, 5)
    return temp, supply, cqload

In [214]:

def modify_file(temp, supply, cqload,lmin,wmin,whichBlock,no_of_inputs):
    with open('NAND_4/NAND.net', 'r') as file:
        data = file.read()
        lines = file.readlines()

    # Modify TEMP
    data = re.sub(r'^dc TEMP .+$', f'dc TEMP {temp:.2f} {temp:.2f} {85}', data, flags=re.MULTILINE)

    # Modify SUPPLY
    data = re.sub(r'^\.PARAM SUPPLY=.+$', f'.PARAM SUPPLY={supply:.2f}', data, flags=re.MULTILINE)

    # Modify Cqload
    data = re.sub(r'Cqload Vout gnd .+f', f'Cqload Vout gnd {cqload:.2f}f', data)
    
    # Modify Lmin
    data = re.sub(r'\.PARAM Lmin=.+$', f'.PARAM Lmin={lmin:.2f}n', data, flags=re.MULTILINE)
    
    # Modify Wmin
    data = re.sub(r'\.PARAM Wmin=.+$', f'.PARAM Wmin={wmin:.2f}n', data, flags=re.MULTILINE)
    
    with open('NAND_4/NAND.net', 'w') as file:
        file.write(data)

    with open('NAND_4/NAND.net', 'r') as file:
        lines = file.readlines()

    # Toggle the lines
    curBlock_Vin,curBlock_meas=0,0
    cnt_Vin,cnt_meas=no_of_inputs,2
    for i in range(len(lines)):
        if lines[i].startswith('*Vin1'):
            curBlock_Vin += 1
        
        if curBlock_Vin==whichBlock and cnt_Vin>=1:
            if lines[i].startswith('*Vin1') or lines[i].startswith('*Vin2') or lines[i].startswith('*Vin3') or lines[i].startswith('*Vin4'):
                lines[i] = lines[i][1:]  # Remove *
                cnt_Vin -= 1
        
        if curBlock_meas//2==whichBlock-1 and cnt_meas>=1:
            if lines[i].startswith('*.measure'):
                lines[i] = lines[i][1:]  # Remove *
                cnt_meas -= 1
        
        if lines[i].startswith('*.measure'):
            curBlock_meas += 1
    
    # Write the modified contents back to the file
    with open('NAND_4/NAND.net', 'w') as file:
        file.writelines(lines)

In [215]:
import re

def modify_file_leakage(temp, supply, lmin, wmin, *Vin_values):
    with open('NAND_4/NAND_leakage.net', 'r') as file:
        data = file.read()
        lines = file.readlines()

    # Modify TEMP
    data = re.sub(r'^dc TEMP .+$', f'dc TEMP {temp:.2f} {temp:.2f} {85}', data, flags=re.MULTILINE)

    # Modify SUPPLY
    data = re.sub(r'^\.PARAM SUPPLY=.+$', f'.PARAM SUPPLY={supply:.2f}', data, flags=re.MULTILINE)
    
    # Modify Lmin
    data = re.sub(r'\.PARAM Lmin=.+$', f'.PARAM Lmin={lmin:.2f}n', data, flags=re.MULTILINE)
    
    # Modify Wmin
    data = re.sub(r'\.PARAM Wmin=.+$', f'.PARAM Wmin={wmin:.2f}n', data, flags=re.MULTILINE)
    
    with open('NAND_4/NAND_leakage.net', 'w') as file:
        file.write(data)

    with open('NAND_4/NAND_leakage.net', 'r') as file:
        lines = file.readlines()
    
    j=0
    for i in range(len(lines)):
        if lines[i].startswith(f'Vin{j+1}'):
            parts = lines[i].split()
            parts[-1] = str(Vin_values[j])
            lines[i] = ' '.join(parts) + '\n'
            j += 1

    # Write the modified contents back to the file
    with open('NAND_4/NAND_leakage.net', 'w') as file:
        file.writelines(lines)


In [216]:
def run_ngspice():
    output = subprocess.check_output(['ngspice', '-b', 'NAND_4/NAND.net']).decode('utf-8')
    delay_lh_match_a = re.search(r'delay_lh\s*=\s*([\d.e+-]+)', output)
    delay_hl_match_a = re.search(r'delay_hl\s*=\s*([\d.e+-]+)', output)
    delay_lh = float(delay_lh_match_a.group(1)) if delay_lh_match_a else None
    delay_hl = float(delay_hl_match_a.group(1)) if delay_hl_match_a else None
    return delay_lh,delay_hl

In [217]:
# Load file content
with open('NAND_4/45nm_HP.pm', 'r') as file:
    original_content = file.read()
    
with open('NAND_4/NAND.net', 'r') as file:
    original_content_net = file.read()
    
# Extract nominal values for parameters
nominal_values = {}
for param in parameters:
    original_value_str, nominal_values[param] = extract_parameter_value(original_content, param)

# Calculate standard deviation (std) for each parameter
std_values = {key: value / 30 for key, value in nominal_values.items()}

# Load file content for Lmin and Wmin parameters
lmin_mean, lmin_std = calculate_mean_std('NAND_4/NAND.net', 'Lmin')
wmin_mean, wmin_std = calculate_mean_std('NAND_4/NAND.net', 'Wmin')

# Number of samples.Change it to 30000
num_samples = 20

# Generate random samples for Lmin and Wmin with Monte Carlo distribution
lmin_samples = np.random.normal(loc=lmin_mean, scale=lmin_std, size=num_samples)
wmin_samples = np.random.normal(loc=wmin_mean, scale=wmin_std, size=num_samples)

# Generate random samples with Monte Carlo distribution
samples = {}
for param, mean in nominal_values.items():
    samples[param] = np.random.normal(loc=mean, scale=std_values[param], size=num_samples)
    
temp_samples=[]
supply_samples=[]
cqload_samples=[]

for i in range(num_samples):
    temp, supply, cqload = generate_random_values()
    temp_samples.append(temp)
    supply_samples.append(supply)
    cqload_samples.append(cqload)

In [218]:
no_of_inputs=4

# Make a copy of the original content for modification
modified_content = original_content

# Replace parameter values in the original file with sampled values
with open('combined_data.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    # Assuming `no_of_inputs` is the number of nodes for which delays are to be calculated
    header_row = ['TEMP', 'pvdd', 'Cqload', 'Lmin', 'Wmin'] + parameters
    for node in range(no_of_inputs):
        header_row += [f'delay_lh_node{chr(97 + node)}', f'delay_hl_node{chr(97 + node)}']
    writer.writerow(header_row)

    for i in range(num_samples):
        # Generate samples and save them in a CSV file
        row = []
        for param in parameters:
            original_value_str, _ = extract_parameter_value(modified_content, param)
            modified_content = modified_content.replace(original_value_str, f'{param} = {samples[param][i]}', 1)
        
        # Save modified content back to the original file
        with open('NAND_4/45nm_HP.pm', 'w') as file:
            file.write(modified_content)
            
        # Calculate delays for each node
        delay_values = []
        for node in range(no_of_inputs):
            lmin = lmin_samples[i]
            wmin = wmin_samples[i]
            temp = temp_samples[i]
            supply = supply_samples[i]
            cqload = cqload_samples[i]
            modify_file(temp, supply, cqload, lmin, wmin,node+1,no_of_inputs)
            
            # Assuming `run_ngspice` returns delay_lh and delay_hl for the current node
            delay_lh, delay_hl = run_ngspice()
            
            # Write original content back to the net file
            with open('NAND_4/NAND.net', 'w') as file:
                file.write(original_content_net)
            
            delay_values.extend([delay_lh, delay_hl])
        
        # Construct the row for the CSV file
        row.extend([temp, supply, cqload, lmin * 1e-9, wmin * 1e-9])
        row.extend(samples[param][i] for param in parameters)
        row.extend(delay_values)
        writer.writerow(row)


In [219]:
# Revert changes back to original values

with open('NAND_4/45nm_HP.pm', 'w') as file:
    file.write(original_content)
    
with open('NAND_4/NAND.net', 'w') as file:
    file.write(original_content_net)

print("Samples generated and saved in 'combined_data.csv'. Original file restored.")

Samples generated and saved in 'combined_data.csv'. Original file restored.


In [220]:
import os
import re
import pandas as pd

def round_to_nearest(value, values_list):
    return min(values_list, key=lambda x: abs(x - value))

def round_gate_voltage(value):
    if value < 0.55:
        return 0
    else:
        return 1.1

def extract_spice_data(filename, ratio_list):
    text = os.popen(f"ngspice {filename}").read()
    voltage_pattern = re.compile(r"v\((\w+)\) = ([\d\.\-\+e]+)")
    current_pattern = re.compile(r"i\((\w+)\) = ([\d\.\-\+e]+)")

    voltages = []
    currents = []

    for match in re.finditer(voltage_pattern, text):
        voltages.append((match.group(1), float(match.group(2))))

    for match in re.finditer(current_pattern, text):
        currents.append((match.group(1), float(match.group(2))))

    rows = []
    
    round_values = [i/20 for i in range(0, 23)]

    # Determine MOSFET type based on the number of rows in the DataFrame
    mosfet_type_list = ['PMOS'] * (len(voltages) // 6) + ['NMOS'] * (len(voltages) // 6)
    # Calculate W_L ratio for PMOS and NMOS
    pmos_ratio, nmos_ratio = ratio_list

    for i in range(len(voltages) // 3):  # Assuming each MOSFET has 3 voltage values
        index = i * 3
        index_current = i * 4
        mosfet_type = mosfet_type_list[i]
        ratio = pmos_ratio if mosfet_type == 'PMOS' else nmos_ratio
        rows.append({'MOSFET Type': mosfet_type,
                     'W_L Ratio': ratio,
                     'Drain Voltage': round_to_nearest(voltages[index][1], round_values),
                     'Gate Voltage': round_gate_voltage(voltages[index+1][1]), # Assuming gate voltage is constant for all transistors
                     'Source Voltage': round_to_nearest(voltages[index+2][1], round_values),
                     'Drain Current': currents[index_current][1],
                     'Gate Current': currents[index_current+1][1], # Assuming gate current is constant for all transistors
                     'Source Current': currents[index_current+2][1],
                     'Body Current': currents[index_current+3][1]})

    df = pd.DataFrame(rows, columns=['MOSFET Type', 'W_L Ratio', 'Drain Voltage', 'Gate Voltage', 'Source Voltage',
                                     'Drain Current', 'Gate Current', 'Source Current', 'Body Current'])

    return df


In [221]:
def calculate_leakage(true_values, drain_voltage, gate_voltage, source_voltage, mos_type,supply):
    # Extracting true currents
    true_drain_current, true_gate_current, true_source_current, true_body_current = true_values
    
    # Checking if the magnitudes of incoming currents match
    
    if mos_type == 'PMOS':
        body_voltage=supply
    else:
        body_voltage=0   
    
    # Calculating outgoing currents based on terminal voltages and direction of current flow
    incoming_currents = []
    if mos_type == 'PMOS':
        # For PMOS, drain is at a lower potential than gate or source
        if (drain_voltage < source_voltage or drain_voltage < body_voltage) and true_drain_current < 0:
            incoming_currents.append(abs(true_drain_current))
        if (gate_voltage < source_voltage or gate_voltage < drain_voltage) and true_gate_current < 0:
            incoming_currents.append(abs(true_gate_current))
        if (source_voltage < drain_voltage or source_voltage < gate_voltage) and true_source_current < 0:
            incoming_currents.append(abs(true_source_current))
        if true_body_current < 0:
            incoming_currents.append(abs(true_body_current))  # Body current always flows out
    elif mos_type == 'NMOS':
        # For NMOS, drain is at a higher potential than gate or source
        if (drain_voltage > source_voltage or drain_voltage > gate_voltage) and true_drain_current < 0:
            incoming_currents.append(abs(true_drain_current))
        if (gate_voltage > source_voltage or gate_voltage > drain_voltage) and true_gate_current < 0:
            incoming_currents.append(abs(true_gate_current))
        if (source_voltage > drain_voltage or source_voltage > gate_voltage) and true_source_current < 0:
            incoming_currents.append(abs(true_source_current))
        if true_body_current < 0:
            incoming_currents.append(abs(true_body_current))  # Body current always flows out
    
    # Calculating leakage current as the sum of outgoing currents
    leakage_current = sum(incoming_currents)
    return leakage_current

In [222]:
def calculate_leakage_for_all_rows(original_list,supply):
    all_leakage_currents = []
    for i in range(len(original_list)):
        leakage_current = calculate_leakage(original_list[i][5:9],
                                            original_list[i][2],
                                            original_list[i][3],
                                            original_list[i][4],
                                            original_list[i][0],supply)
        all_leakage_currents.append(leakage_current)
        
    return sum(all_leakage_currents)

In [223]:
# Load file content
with open('NAND_4/45nm_HP.pm', 'r') as file:
    original_content = file.read()

with open('NAND_4/NAND_leakage.net', 'r') as file:
    original_content_leakage = file.read()

In [224]:
from itertools import product

# Define the number of inputs
no_of_inputs = 4  # You can change this value as needed

# Make a copy of the original content for modification
modified_content = original_content

# Replace parameter values in the original file with sampled values
with open('combined_data1.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    header_row = ['Vin_' + chr(65 + i) for i in range(no_of_inputs)] + ['TEMP', 'pvdd', 'Lmin', 'Wmin'] + parameters + ['Leakage_power']
    writer.writerow(header_row)

    for i in range(num_samples):
        # Generate samples and save them in a CSV file
        for param in parameters:
            original_value_str, _ = extract_parameter_value(modified_content, param)
            modified_content = modified_content.replace(original_value_str, f'{param} = {samples[param][i]}', 1)
        
        # Save modified content back to the original file
        with open('NAND_4/45nm_HP.pm', 'w') as file:
            file.write(modified_content)
        
        lmin = lmin_samples[i]
        wmin = wmin_samples[i]
        temp = temp_samples[i]
        supply = supply_samples[i]
        
        # Iterate over all possible combinations of input voltages
        for Vin_values in product(range(2), repeat=no_of_inputs):
            row = []
            input_values = [(supply if val == 1 else 0) for val in Vin_values]
            
            modify_file_leakage(temp, supply, lmin, wmin, *input_values)
            NAND_ratio = [2,2]
            
            # Extracting voltage and current values from executing netlist
            file = "NAND_4/NAND_leakage.net"
            file_df_extracted = extract_spice_data(file, NAND_ratio)
            file_extracted = file_df_extracted.values.tolist()
            leakage_currents = calculate_leakage_for_all_rows(file_extracted, supply)
            
            row.extend(Vin_values)
            row.extend([temp, supply, lmin * 1e-9, wmin * 1e-9])
            row.extend(samples[param][i] for param in parameters)
            row.extend([leakage_currents * (supply + sum(Vin_values))])
            writer.writerow(row)




Note: Starting dynamic gmin stepping
Trying gmin =   1.0000E-03 Note: One successful gmin step
Trying gmin =   1.0000E-04 Note: One successful gmin step
Trying gmin =   1.0000E-05 Note: One successful gmin step
Trying gmin =   3.1623E-06 Note: One successful gmin step
Trying gmin =   5.6234E-07 Note: One successful gmin step
Trying gmin =   5.6234E-08 Note: One successful gmin step
Trying gmin =   5.6234E-09 Note: One successful gmin step
Trying gmin =   5.6234E-10 Note: One successful gmin step
Trying gmin =   5.6234E-11 Note: One successful gmin step
Trying gmin =   5.6234E-12 Note: One successful gmin step
Trying gmin =   5.6234E-13 Note: One successful gmin step
Trying gmin =   5.6234E-14 Note: One successful gmin step
Trying gmin =   5.6234E-15 Note: One successful gmin step
Trying gmin =   5.6234E-16 Note: One successful gmin step
Trying gmin =   5.6234E-17 Note: One successful gmin step
Trying gmin =   5.6234E-18 Note: One successful gmin step
Trying gmin =   5.6234E-19 Note: 

In [225]:
# Revert changes back to original values

with open('NAND_4/45nm_HP.pm', 'w') as file:
    file.write(original_content)
    
with open('NAND_4/NAND_leakage.net', 'w') as file:
    file.write(original_content_leakage)

print("Samples generated and saved in 'combined_data1.csv'. Original file restored.")

Samples generated and saved in 'combined_data1.csv'. Original file restored.
